In [ ]:
# default_exp bcml_model

# bcml_model

> Core class representing a binary classification (BC) machine learning (ML) model

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# hide
import numpy as np
import sklearn.metrics as skm
import scipy.interpolate
import scipy.optimize
import pandas as pd
import joblib

In [ ]:
# export
class mlbc_model:
    """
    Represents a ML binary classification model.
    """
    
    def __init__(self, model, train, test, num_bgs=3):
        self.model = model
        self.train = train
        self.test = test
        self.num_bgs = num_bgs
        self.train_preds = train[:,:-1]
        self.train_labels = np.where(train[:,-1]==1, np.ones_like(train[:,-1]), np.zeros_like(train[:,-1]))
        self.test_preds = test[:,:-1]
        self.test_labels = np.where(test[:,-1]==1, np.ones_like(test[:,-1]), np.zeros_like(test[:,-1]))
        self.test_bgs = [self.test[self.test[:,-1] == -i] for i in range(1,self.num_bgs+1)]
        self.test_bgs_preds = [data[:,:-1] for data in self.test_bgs]
        self.test_sigs = self.test[self.test[:,-1] == 1]
        self.test_sigs_preds = self.test_sigs[:,:-1]
    
    def fit(self, preds=None, labels=None):
        preds = preds if preds is not None else self.train_preds
        labels = labels if labels is not None else self.train_labels
        self.model.fit(preds, labels)
        
    def predict_proba(self, preds=None):
        preds = preds if preds is not None else self.test_preds
        try:
            return self.model.predict_proba(preds)[:,1]
        except:
            print("self.model doesn't have a predict_proba function")
            
    def predict(self, preds=None, threshold=None):
        preds = preds if preds is not None else self.test_preds
        if threshold is not None:
            probs = self.model.predict_proba(preds)[:,1]
            return np.where(probs > threshold, np.ones_like(probs), np.zeros_like(probs))
        else:
            try:
                return self.model.predict(preds)
            except:
                print("self.model doesn't have a predict function")
                
    def predict_hist(self, preds=None, labels=None, num_bins=100, sepbg=False, sig_norm=1, bg_norm=1, dataframe=False):
        preds = preds if preds is not None else self.test_preds
        labels = labels if labels is not None else self.test_labels
        predictions = self.predict_proba(preds)
        sig_bins, bin_edges = np.histogram(predictions[labels==1], bins=num_bins, density=True)
        sig_bins *= sig_norm
        if sepbg:
            bg_norms = bg_norm
            bg_binss = [
                bg_norm * np.histogram(predictions[labels==-i], bins=num_bins, density=True)[0] 
                for i, bg_norm in enumerate(bg_norms)]
            if dataframe:
                return pd.DataFrame(data=[bin_edges, sig_bins] + bg_binss, columns=['Bin Edges', 'Signal'] + ['Background {}'.format(i) for i in range(1, self.num_bgs+1)])
            else:
                return [bin_edges, sig_bins] + bg_binss
        else:
            bg_bins = np.histogram(predictions[labels!=1], bins=num_bins, density=True)
            if dataframe:
                return pd.DataFrame(data=[bin_edges, sig_bins, bg_bins], columns=['Bin Edges', 'Signal', 'Background'])
            else:
                return [bin_edges, sig_bins, bg_bins]
            
    def feature_importance(model):
        return model.feature_importances_
    
    def sorted_feature_importance(features, importances):
        ranked_indices = np.argsort(-np.abs(importances))
        return [[features[i], importances[i]] for i in ranked_indices]
        
    def accuracy(self, preds=None, labels=None, threshold=None):
        preds = preds if preds is not None else self.test_preds
        labels = labels if labels is not None else self.test_labels
        predictions = self.predict(preds=preds, threshold=threshold)
        return len(preds) - np.sum(np.abs(predictions - labels)) / len(preds)
        
    def conf_matrix(self, predictions=None, labels=None):
        predictions = predictions if predictions is not None else self.predict(self.test_preds)
        labels = labels if np.any(labels) is not None else self.test_labels
        return skm.confusion_matrix(labels, predictions, labels=[0,1])
    
    def tpr_cm(self, conf_matrix):
        return conf_matrix[1,1]/np.sum(conf_matrix[1])
        
    def fpr_cm(self, conf_matrix):
        return conf_matrix[0,1]/np.sum(conf_matrix[0])
    
    def tpr(self, predictions=None, labels=None):
        """true positive rate: correctly identified signal / all signal"""
        predictions = predictions if predictions is not None else self.predict(self.test_preds)
        labels = labels if labels is not None else self.test_labels
        return self.tpr_cm(self.conf_matrix(predictions, labels))
    
    def fpr(self, predictions=None, labels=None):
        """false positive rate: misidentified signal / all background"""
        predictions = predictions if predictions is not None else self.predict(self.test_preds)
        labels = labels if labels is not None else self.test_labels
        return self.fpr_cm(self.conf_matrix(predictions, labels))
    
    def significance(self, signal, background, tpr=None, fpr=None, sepbg=False):
        tpr = tpr if tpr is not None else self.tpr()
        if sepbg:
            fprs = fpr if fpr is not None else [
                self.fpr(self.predict(pred), label) for pred, label in zip(
                    self.test_bgs_preds, np.zeros(np.sum([len(x) for x in self.test_bgs_preds])))]
            fprXbackground = np.sum(np.multiply(fprs, background), axis=-1)
            return signal * tpr / np.sqrt(signal * tpr + fprXbackground + 1e-10)
        else:
            fpr = fpr if fpr is not None else self.fpr()
            return signal * tpr / np.sqrt(signal * tpr + background * fpr + 1e-10)
    
    def newvar2thresh(self, newvar):
        return 1 - np.power(10, newvar)
    
    def thresh2newvar(self, thresh):
        return np.log10(1 - thresh)
    
    def max_allowable_threshold(self):
        newvars = np.concatenate((np.linspace(-8, -2, 10, endpoint=False), np.linspace(-2, 0, 51, endpoint=False)))
        probs = self.model.predict_proba(self.test_pred)[:,1]
        predicts = np.array(
            [np.where(probs > self.newvar2thresh(newvar),
                      np.ones_like(probs), np.zeros_like(probs)) for newvar in newvars])
        num_sig = [np.sum(predict[(predict == 1) & (self.test_label == 1)]) for predict in predicts]
        f = scipy.interpolate.interp1d(num_sig, newvars, kind='cubic')
        return self.newvar2thresh(f(25))
    
    def get_tprs_fprs(self, preds=None, labels=None, sepbg=False):
        """
        newvar = log_10(1 - threshold)
        threshold = 1 - 10**newvar
        0 < threshold < 1
        ??? < new_var < 0
        """
        
        # setting up variables
        preds = preds if preds is not None else ([self.test_sigs_preds] + self.test_bgs_preds if sepbg else self.test_preds)
        labels = labels if labels is not None else ([np.ones_like(preds[0][:,0])] + [np.zeros_like(bg_preds[:,0]) for bg_preds in preds[1:]] if sepbg else self.test_labels)
        min_newvar, max_newvar = [-8, 0]
        newvars = np.concatenate((np.linspace(min_newvar, -2, 10, endpoint=False), np.linspace(-2, max_newvar, 51, endpoint=False)))
        
        # computing significance as a function of threshold
        if sepbg:
            predss = preds
            labelss = labels
            probss = [self.predict_proba(preds) for preds in predss]
            predictionsss = np.array(
                [[np.where(probs > self.newvar2thresh(newvar), 
                          np.ones_like(probs), np.zeros_like(probs)) for probs in probss] for newvar in newvars])
            sig_conf_matrices = [
                self.conf_matrix(predictions=predictionss[0], labels=labelss[0]) for predictionss in predictionsss]
            bg_conf_matricess = [
                [self.conf_matrix(predictions=predictions, labels=labelss[i+1]) for i, predictions in enumerate(predictionss[1:])] for predictionss in predictionsss]
            tprs = np.array([self.tpr_cm(conf_matrix) for conf_matrix in sig_conf_matrices])
            fprss = np.array([[self.fpr_cm(conf_matrix) for conf_matrix in conf_matrices] for conf_matrices in bg_conf_matricess])
            return [newvars, tprs, fprss, probss]
        else:
            probs = self.predict_proba(preds)
            predictionss = np.array(
                [np.where(probs > self.newvar2thresh(newvar), 
                          np.ones_like(probs), np.zeros_like(probs)) for newvar in newvars])
            conf_matrices = [self.conf_matrix(predictions=predictions, labels=labels) for predictions in predictionss]
            tprs = [self.tpr_cm(conf_matrix) for conf_matrix in conf_matrices]
            fprs = [self.fpr_cm(conf_matrix) for conf_matrix in conf_matrices]
            return [newvars, tprs, fprs, probs]
        
    def best_threshold(self, signal, background, preds=None, labels=None, sepbg=False):
        """
        newvar = log_10(1 - threshold)
        threshold = 1 - 10**newvar
        0 < threshold < 1
        ??? < new_var < 0
        """
        
        preds = preds if preds is not None else ([self.test_sigs_preds] + self.test_bgs_preds if sepbg else self.test_preds)
        labels = labels if labels is not None else ([np.ones_like(preds[0][:,0])] + [np.zeros_like(bg_preds[:,0]) for bg_preds in preds[1:]] if sepbg else self.test_labels)
        newvars, tprs, fprs, probs = self.get_tprs_fprs(preds, labels, sepbg)
        significances = -self.significance(signal, background, tprs, fprs, sepbg=sepbg)

        # interpolating significance as a function of threshold, then maximizing
        f = scipy.interpolate.interp1d(newvars, significances, kind='cubic')
        res = scipy.optimize.minimize(f, [-2], bounds=[(newvars[0] + 1e-1, newvars[-1] - 1e-1)])
        
        # computing significance, tpr, fpr for optimized threshold
        best_threshold = self.newvar2thresh(res.x[0])
        if sepbg:
            probss = probs
            labelss = labels
            fprss = fprs
            best_predictss = [np.where(probs > best_threshold, np.ones_like(probs), np.zeros_like(probs)) for probs in probss]
            sig_conf_matrix = self.conf_matrix(predictions=best_predictss[0], labels=labelss[0])
            bg_conf_matrices = [self.conf_matrix(predictions=best_predicts, labels=labelss[i+1]) for i, best_predicts in enumerate(best_predictss[1:])]
            tpr = self.tpr_cm(sig_conf_matrix)
            fprs = [self.fpr_cm(conf_matrix) for conf_matrix in bg_conf_matrices]
            best_sig = self.significance(signal, background, tpr, fprs, sepbg=sepbg)
            return [best_threshold, best_sig, tpr, fprs, tprs, fprss]
        else:
            best_predictss = np.where(probs > best_threshold, np.ones_like(probs), np.zeros_like(probs))
            conf_matrix = self.conf_matrix(predictions=best_predict)
            tpr = self.tpr_cm(conf_matrix)
            fpr = self.fpr_cm(conf_matrix)
            best_sig = self.significance(signal, background, tpr, fpr, sepbg=sepbg)
            return [best_threshold, best_sig, tpr, fpr, tprs, fprs]
    
    def req_sig_cs(self, lumi, bg_cs, tpr, fpr, sig=5):
        conv = 10**15 / 10**12
        coef = [-tpr**2 * lumi**2 * conv**2, sig**2 * tpr * lumi * conv, sig**2 * fpr * bg_cs * lumi * conv]
        return np.amax(np.roots(coef))
        
    def save_model(self, filename):
        joblib.dump(self.model, filename + '.joblib')
        
    def refresh_model(model):
        return sigbg_model(model.model, model.train, model.test)

## Creating a `bcml_model` instance

To create a new `bcml_model` instance, three things are required. 

First, you need `model`, a machine learning binary classification model: this library was designed to work with `scikit-learn` classifiers, such as `sklearn.linear_model.LogisticRegression`, `sklearn.ensemble.RandomForestClassifier`, or `sklearn.ensemble.GradientBoostingClassifier`.

Second, you need `train`, a training data set. This should be a `numpy` array of shape $N \times (M + 1)$ if you have $N$ training data points and $M$ features. The first $M$ columns of `train` should contain your features, while the final column is your label: $1$ denotes signal, while $0$ or a strictly negative integer denotes background. Negative integers can be used to denote different kinds of background: i.e., $-2$ would denote the background of type $2$.

Third, you need `test`, a testing data set. This should be a `numpy` array of shape $N' \times (M + 1)$ formatted the same as `train`. Typically, $N' \approx \frac{1}{3}N$.

If you utilize the negative integer labelling of background data points, you should pass `num_bgs` as well, denoting the number of different backgrounds you're using (i.e., your backgrounds labels are elements of $\{-\text{num_bgs}, \dots, -1\}$).

## `bcml_model` attributes

In addition to the arguments passed, a `bcml_model` instance has the following attributes.

`train_preds`: training predictors, $N \times M$ `numpy` array, the first $M$ columns of train.

`train_labels`: training labels, `numpy` array of length $N$, final column of train (with negative integers mapped to $0$)

`test_preds`: test predictors, $N' \times M$ `numpy` array, first $M$ columns of test

`test_labels`: test labels, `numpy` array of length $N'$, final column of test (with negative integers mapped to $0$)

`test_bgs`: background test data split by background, list of $N_i \times (M + 1)$ `numpy` arrays such that the $i$th element contains all test data points of background type $i$ (i.e., $1 \leq i \leq$ `num_bgs` and $\sum_i N_i = N' - $ # of signal data points)

`test_bgs_preds`: background test predictors split by background, list of $N_i \times M$ `numpy` arrays

`test_sigs`: signal test data, $\overline{N} \times (M + 1)$ `numpy` array (i.e., $\overline{N} + \sum_i N_i = N'$)

`test_sigs_preds`: signal test data predictors, $\overline{N} \times M$ `numpy` array